# Desarrollo del script de ejecución por el usuario

## Procesamiento datos

In [ ]:
from LIACEI_workflow.data.DinamicaMolecular import DinamicaMolecular, ElementoQuimico
from LIACEI_workflow.utils.utils import crear_carpeta
from LIACEI_workflow.codes.torchmd_net.utils import guardar_a_TorchMDNet

In [ ]:
# cargar y convertir datos .data
base_de_datos=DinamicaMolecular.cargar_desde_data("input/input.data")

In [ ]:
for frame in base_de_datos.frames.values():
    # Calcular la suma de las energías atómicas de los elementos
    energia_total = sum(ElementoQuimico(e).energia_atomica() for e in frame.elementos)
    
    # Restar la energía atómica total de la energía actual del frame
    frame.energia -= energia_total  # Actualizar directamente el atributo

In [ ]:
crear_carpeta("transform")

In [ ]:
guardar_a_TorchMDNet(base_de_datos, "transform/input_torchmd-net.h5")

In [ ]:
crear_carpeta("train")

```bash
torchmd-train --conf input/config.yaml
```

## Generar inferencias

In [ ]:
from LIACEI_workflow.data.DinamicaMolecular import DinamicaMolecular
from LIACEI_workflow.codes.torchmd_net.utils import (
    cargar_desde_TorchMDNet,
    generar_inferencias,
    obtener_indices_test,
    obtener_ruta_modelo_entrenado
)

In [ ]:
ruta_modelo = obtener_ruta_modelo_entrenado("train")

In [ ]:
base_de_datos = cargar_desde_TorchMDNet("transform/input_torchmd-net.h5")

In [ ]:
test_idx = obtener_indices_test("train/splits.npz")

In [ ]:
subconjunto_test = DinamicaMolecular.subconjunto(base_de_datos, test_idx)

In [ ]:
inferencias = generar_inferencias(subconjunto_test,ruta_modelo)

In [ ]:
# Guardar Inferencias
DinamicaMolecular.guardar_a_hdf5(inferencias,"train/output_LIACEI.h5")

## Generación datos para graficas

In [ ]:
from LIACEI_workflow.data.DinamicaMolecular import Utils
from LIACEI_workflow.data.DinamicaMolecular import DinamicaMolecular
from LIACEI_workflow.utils.utils import crear_carpeta
from LIACEI_workflow.codes.torchmd_net.utils import (
    cargar_desde_TorchMDNet,
    guardar_metricas,
    obtener_indices_test
)

In [ ]:
# Cargar referencias
referencias = cargar_desde_TorchMDNet("transform/input_torchmd-net.h5")
test_idx = obtener_indices_test("train/splits.npz")
referencias = DinamicaMolecular.subconjunto(referencias, test_idx)

In [ ]:
# Cargar inferencias
inferencias = DinamicaMolecular.cargar_desde_hdf5("train/output_LIACEI.h5")

In [ ]:
crear_carpeta("analysis")

In [ ]:
guardar_metricas("train/metrics.csv","analysis/epoch_vs_loss.data")

In [ ]:

Utils.comparar_frames(referencias, inferencias)

In [ ]:

Utils.calcular_angulos_y_guardar(referencias, inferencias)

In [ ]:

Utils.calcular_modulos_fuerza_y_guardar(referencias, inferencias)

In [ ]:

Utils.guardar_energia_referencia_vs_inferencia(referencias, inferencias)

## Graficas

In [2]:
from LIACEI_workflow.analysis.DinamicaMolecular import Graficas_genericas

In [3]:
Graficas_genericas.grafica_epoch_vs_loss()

In [4]:
Graficas_genericas.grafica_ref_vs_inf_energy()

In [5]:
Graficas_genericas.grafica_ref_vs_inf_energy_hist_2d()

In [6]:
Graficas_genericas.grafica_ref_vs_inf_energy_hist_3d()

In [7]:
Graficas_genericas.grafica_ref_vs_inf_energy_hist()

In [8]:
Graficas_genericas.grafica_ref_vs_inf_energy_abs_dif()

In [9]:
Graficas_genericas.graficas_ref_vs_inf_forces_angles_abs_dif()

/workspace/refactorizacion/LIACEI_workflow/analysis/DinamicaMolecular.py:277: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  datos = pd.read_csv(path_data_file, header=None, skiprows=1, delim_whitespace=True)
/workspace/refactorizacion/LIACEI_workflow/analysis/DinamicaMolecular.py:277: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  datos = pd.read_csv(path_data_file, header=None, skiprows=1, delim_whitespace=True)
/workspace/refactorizacion/LIACEI_workflow/analysis/DinamicaMolecular.py:277: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  datos = pd.read_csv(path_data_file, header=None, skiprows=1, delim_whitespace=True)
/workspace/refactorizacion/LIACEI_workflow/analysis/DinamicaMolecular.py:277: Futu

In [10]:
Graficas_genericas.graficas_ref_vs_inf_forces_modules()

In [11]:
Graficas_genericas.graficas_ref_vs_inf_forces_modules_hist_2d()

In [12]:
Graficas_genericas.graficas_ref_vs_inf_forces_modules_hist_3d()

In [13]:
Graficas_genericas.graficas_ref_vs_inf_forces_modules_hist()